In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
import os

In [3]:
from sklearn.metrics import roc_auc_score,accuracy_score

In [4]:
os.chdir('../output_data')

In [5]:
df = pd.read_csv('train_v05.csv',header=0)

In [6]:
df.set_index('PassengerId',inplace=True)

In [7]:
df.shape

(712, 10)

# Inferential Model

In [8]:
scale = pickle.load(open('scale.pickle','rb'))

In [9]:
pca = pickle.load(open('pca.pickle','rb'))

In [10]:
x = df.drop(['Survived'],axis=1)

In [11]:
tuned_parameters = {'C':np.logspace(-2,3,6),'solver':['lbfgs','liblinear']}
model = GridSearchCV(LogisticRegression(max_iter=1000),tuned_parameters,scoring='roc_auc',
                    cv=5,verbose=60)

In [12]:
y = np.array(df['Survived'])

In [13]:
model.fit(x,y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.790886 -   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.948606 -   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.837703 -   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.899300 -   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[CV] C=0.01, solver=lbfgs .........................

[CV] .................. C=100.0, solver=lbfgs, score=0.947774 -   0.0s
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:    0.6s remaining:    0.0s
[CV] C=100.0, solver=lbfgs ...........................................
[CV] .................. C=100.0, solver=lbfgs, score=0.836246 -   0.0s
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:    0.7s remaining:    0.0s
[CV] C=100.0, solver=lbfgs ...........................................
[CV] .................. C=100.0, solver=lbfgs, score=0.846089 -   0.0s
[Parallel(n_jobs=1)]: Done  44 out of  44 | elapsed:    0.7s remaining:    0.0s
[CV] C=100.0, solver=lbfgs ...........................................
[CV] .................. C=100.0, solver=lbfgs, score=0.922813 -   0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.7s remaining:    0.0s
[CV] C=100.0, solver=liblinear .......................................
[CV] .............. C=100.0, solver=liblinear, score=0.789638 -   0.0s
[Parallel(n_jobs=1)]: Done  46 out of  46

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03]), 'solver': ['lbfgs', 'liblinear']},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=60)

In [14]:
model = model.best_estimator_

In [15]:
os.chdir('../Models/Inferential_models')

In [16]:
pickle.dump(model,open('LR.pickle','wb'))

In [17]:
model_columns = df.drop(['Survived'],axis=1).columns

In [18]:
os.chdir('../../pickles')

In [19]:
pickle.dump(model_columns,open('model_columns.pickle','wb'))

# Predictive Model

In [33]:
os.chdir('../output_data')

In [21]:
df_pca = pd.read_csv('train_pca.csv',header=0)

In [22]:
df_pca.set_index('PassengerId',inplace=True)

In [30]:
y=np.array(df_pca['Survived'])

In [24]:
tuned_parameters = {'C':np.logspace(-2,3,6),'solver':['lbfgs','liblinear']}
model = GridSearchCV(LogisticRegression(max_iter=1000),tuned_parameters,scoring='roc_auc',
                    cv=5,verbose=60)

In [25]:
os.chdir('../Models/pca_models')

In [26]:
x = df_pca.drop(['Survived'],axis=1)

In [31]:
model.fit(x,y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.787141 -   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.966916 -   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.832709 -   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.01, solver=lbfgs ............................................
[CV] ................... C=0.01, solver=lbfgs, score=0.848633 -   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[CV] C=0.01, solver=lbfgs .........................

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03]), 'solver': ['lbfgs', 'liblinear']},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=60)

In [36]:
model = model.best_estimator_

In [37]:
pickle.dump(model,open('LR.pickle','wb'))

# Random Forest

# Inferential Model

In [39]:
os.chdir('../../pickles')

In [40]:
os.chdir('../Models/Inferential_models')

In [41]:
tuned_parameters={'max_features':[None,'sqrt','log2'],'min_samples_leaf':[i*.01 for i in range(1,11)]}
model = GridSearchCV(RandomForestClassifier(n_jobs=-1,n_estimators=1000,oob_score=True,random_state=42),tuned_parameters,cv=5,scoring='roc_auc',verbose=1000)

In [42]:
x = df.drop(['Survived'],axis=1)

In [43]:
y = np.array(df['Survived'])

In [44]:
model.fit(x,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.575218 -   3.7s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.006242 -   3.7s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.4s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.047233 -   3.7s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   11.1s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.449438 -   3.7s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   14.8s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 

[CV]  max_features=None, min_samples_leaf=0.08, score=0.608198 -   3.5s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  2.3min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.537312 -   3.5s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  2.4min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.852058 -   3.4s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  2.4min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.761340 -   4.1s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:  2.5min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.721182 -   3.4s
[Parallel(n_jobs=1)]: Done  42 out o

[CV]  max_features=sqrt, min_samples_leaf=0.05, score=0.862672 -   3.9s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  4.6min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.746151 -   3.6s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:  4.7min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.822305 -   3.7s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:  4.7min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.740533 -   3.7s
[Parallel(n_jobs=1)]: Done  78 out of  78 | elapsed:  4.8min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.603774 -   3.7s
[Parallel(n_jobs=1)]: Done  79 out o

[CV]  max_features=log2, min_samples_leaf=0.03, score=0.805035 -   3.6s
[Parallel(n_jobs=1)]: Done 112 out of 112 | elapsed:  6.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.719309 -   3.8s
[Parallel(n_jobs=1)]: Done 113 out of 113 | elapsed:  6.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.618826 -   3.6s
[Parallel(n_jobs=1)]: Done 114 out of 114 | elapsed:  7.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.863422 -   3.5s
[Parallel(n_jobs=1)]: Done 115 out of 115 | elapsed:  7.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.04 ........................
[CV]  max_features=log2, min_samples_leaf=0.04, score=0.746151 -   3.9s
[Parallel(n_jobs=1)]: Done 116 out o

[CV]  max_features=log2, min_samples_leaf=0.1, score=0.590206 -   3.6s
[Parallel(n_jobs=1)]: Done 149 out of 149 | elapsed:  9.1min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.1 .........................
[CV]  max_features=log2, min_samples_leaf=0.1, score=0.805210 -   3.8s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  9.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  9.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc',
       verbose=1000)

In [55]:
model.best_score_

0.7694891430526777

In [56]:
final_model = model.best_estimator_

In [57]:
final_model.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=0.07, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [58]:
pickle.dump(final_model,open('RF.pickle','wb'))

# Predictive Model

In [59]:
os.chdir('../../pickles')

In [60]:
os.chdir('../Models/pca_models')

In [61]:
x = np.array(df_pca.drop(["Survived"],axis=1))

In [62]:
y =np.array(df_pca['Survived'])

In [63]:
tuned_parameters={'max_features':[None,'sqrt','log2'],'min_samples_leaf':[i*.01 for i in range(1,11)]}
model = GridSearchCV(RandomForestClassifier(n_jobs=-1,n_estimators=1000,oob_score=True,random_state=42),tuned_parameters,cv=5,scoring='roc_auc',verbose=1000)

In [64]:
model.fit(x,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.523304 -   3.6s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.391178 -   3.7s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.3s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.426550 -   3.8s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   11.2s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.699809 -   3.9s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   15.0s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 

[CV]  max_features=None, min_samples_leaf=0.08, score=0.729193 -   3.5s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  2.3min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.764681 -   3.7s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  2.3min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.914987 -   3.6s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  2.4min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.566271 -   3.6s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:  2.5min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.865585 -   3.6s
[Parallel(n_jobs=1)]: Done  42 out o

[CV]  max_features=sqrt, min_samples_leaf=0.05, score=0.900300 -   3.7s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  4.5min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.519455 -   3.6s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:  4.6min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.844569 -   3.7s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:  4.6min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.786101 -   3.6s
[Parallel(n_jobs=1)]: Done  78 out of  78 | elapsed:  4.7min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.782913 -   3.7s
[Parallel(n_jobs=1)]: Done  79 out o

[CV]  max_features=log2, min_samples_leaf=0.03, score=0.822514 -   3.6s
[Parallel(n_jobs=1)]: Done 112 out of 112 | elapsed:  6.8min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.713275 -   3.8s
[Parallel(n_jobs=1)]: Done 113 out of 113 | elapsed:  6.8min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.748993 -   3.6s
[Parallel(n_jobs=1)]: Done 114 out of 114 | elapsed:  6.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.912307 -   3.6s
[Parallel(n_jobs=1)]: Done 115 out of 115 | elapsed:  6.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.04 ........................
[CV]  max_features=log2, min_samples_leaf=0.04, score=0.520183 -   3.6s
[Parallel(n_jobs=1)]: Done 116 out o

[CV]  max_features=log2, min_samples_leaf=0.1, score=0.783231 -   3.7s
[Parallel(n_jobs=1)]: Done 149 out of 149 | elapsed:  9.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.1 .........................
[CV]  max_features=log2, min_samples_leaf=0.1, score=0.838551 -   3.7s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  9.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  9.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc',
       verbose=1000)

In [65]:
model.best_score_

0.7694891430526777

In [66]:
final_model = model.best_estimator_

In [67]:
final_model.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=0.07, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [68]:
pickle.dump(final_model,open('RF.pickle','wb'))

# Gradient Boosting Model

# Inferential Model

In [69]:
os.chdir('../../pickles')

In [70]:
os.chdir('../Models/Inferential_models')

In [71]:
x = df.drop(['Survived'],axis=1)

In [72]:
y = np.array(df['Survived'])

In [73]:
tuned_parameters={'n_estimators':[100*i for i in range(1,11)]}
model = GridSearchCV(GradientBoostingClassifier(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='roc_auc',verbose=50)

In [74]:
model.fit(x,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.541303 -   0.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.047649 -   0.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.012901 -   0.2s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.022896 -   0.2s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[CV] n_estimators=100 .............................

[CV] ....................... n_estimators=800, score=0.009779 -   1.2s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:   28.5s remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.048548 -   1.2s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   29.6s remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.828902 -   1.3s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   31.0s remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.669476 -   1.3s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:   32.3s remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.026633 -   1.5s
[Parallel(n_jobs=1)]: Done  42 out of  42

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=50)

In [75]:
model.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=800,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False)

In [76]:
tuned_parameters={'max_depth':[8,10,12,15,20],'min_samples_leaf':[.01*i for i in range(1,11)],'subsample':[.8,.9,1]
                 ,'max_features':[None,'sqrt','log2']}
model = GridSearchCV(GradientBoostingClassifier(n_estimators=800,learning_rate=.1,random_state=42),tuned_parameters,cv=5,scoring='roc_auc',verbose=2500)

In [77]:
model.fit(x,y)

Fitting 5 folds for each of 450 candidates, totalling 2250 fits
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.626821 -   1.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.014357 -   1.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.007491 -   1.1s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.4s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.049184 -   1

[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.006034 -   0.8s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:   32.9s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.052576 -   0.8s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:   33.7s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.839408 -   0.8s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   34.4s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03, score=0.701935 -   0.8s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   35.2s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.739492 -   0.7s
[Parallel(n_jobs=1)]: Done  66 out of  66 | elapsed:   57.5s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.000000 -   0.7s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:   58.2s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.006450 -   0.7s
[Parallel(n_jobs=1)]: Done  68 out of  68 | elapsed:   58.8s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.013992 -   0.7s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:   59.5s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07, score=0.012084 -   0.6s
[Parallel(n_jobs=1)]: Done  99 out of  99 | elapsed:  1.3min remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07, score=0.865888 -   0.6s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07, score=0.761340 -   0.5s
[Parallel(n_jobs=1)]: Done 101 out of 101 | elapsed:  1.3min remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07, score=0.000000 -   0.5s
[Parallel(n_jobs=1)]: Done 102 out of 102 | elapsed:  1.3min remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_l

[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.006866 -   0.5s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:  1.6min remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.015397 -   0.4s
[Parallel(n_jobs=1)]: Done 133 out of 133 | elapsed:  1.6min remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.013356 -   0.4s
[Parallel(n_jobs=1)]: Done 134 out of 134 | elapsed:  1.6min remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.841981 -   0.4s
[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  1.6min remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=

...

[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03, score=0.111735 -   0.6s
[Parallel(n_jobs=1)]: Done 2142 out of 2142 | elapsed: 21.4min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03, score=0.068248 -   0.6s
[Parallel(n_jobs=1)]: Done 2143 out of 2143 | elapsed: 21.4min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03, score=0.155607 -   0.5s
[Parallel(n_jobs=1)]: Done 2144 out of 2144 | elapsed: 21.4min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03, score=0.821184 -   0.5s
[Parallel(n_jobs=1)]: Done 2145 out of 2145 | elapsed: 21.4min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, m

[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.05, score=0.033072 -   0.4s
[Parallel(n_jobs=1)]: Done 2174 out of 2174 | elapsed: 21.7min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.05 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.05, score=0.890759 -   0.4s
[Parallel(n_jobs=1)]: Done 2175 out of 2175 | elapsed: 21.7min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06, score=0.732834 -   0.5s
[Parallel(n_jobs=1)]: Done 2176 out of 2176 | elapsed: 21.7min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06, score=0.002081 -   0.5s
[Parallel(n_jobs=1)]: Done 2177 out of 2177 | elapsed: 21.7min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_dep

[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.08, score=0.734290 -   0.4s
[Parallel(n_jobs=1)]: Done 2206 out of 2206 | elapsed: 21.9min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.08 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.08, score=0.001040 -   0.4s
[Parallel(n_jobs=1)]: Done 2207 out of 2207 | elapsed: 21.9min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.08 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.08, score=0.021848 -   0.4s
[Parallel(n_jobs=1)]: Done 2208 out of 2208 | elapsed: 21.9min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.08 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.08, score=0.013356 -   0.4s
[Parallel(n_jobs=1)]: Done 2209 out of 2209 | elapsed: 21.9min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, m

[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.1, score=0.020599 -   0.4s
[Parallel(n_jobs=1)]: Done 2238 out of 2238 | elapsed: 22.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.1 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.1, score=0.009964 -   0.4s
[Parallel(n_jobs=1)]: Done 2239 out of 2239 | elapsed: 22.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.1 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.1, score=0.865030 -   0.4s
[Parallel(n_jobs=1)]: Done 2240 out of 2240 | elapsed: 22.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.1 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.1, score=0.722222 -   0.4s
[Parallel(n_jobs=1)]: Done 2241 out of 2241 | elapsed: 22.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_dept

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=800,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'subsample': [0.8, 0.9, 1], 'max_depth': [8, 10, 12, 15, 20], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc',
       verbose=2500)

In [78]:
model.best_score_

0.45429840917586356

In [79]:
model.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=20,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=800,
              presort='auto', random_state=42, subsample=1, verbose=0,
              warm_start=False)

In [80]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingClassifier(max_depth=20,max_features='sqrt',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=400,learning_rate=.2),tuned_parameters,cv=5,scoring='roc_auc')

In [81]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=20,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=400,
              presort='auto', random_state=42, subsample=1, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [82]:
model.best_score_

0.43166311575806554

In [83]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingClassifier(max_depth=20,max_features='sqrt',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=200,learning_rate=.4),tuned_parameters,cv=5,scoring='roc_auc')

In [84]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.4, loss='deviance', max_depth=20,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=42, subsample=1, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [85]:
model.best_score_

0.4474364131068426

In [86]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingClassifier(max_depth=20,max_features='sqrt',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=1600,learning_rate=.05),tuned_parameters,cv=5,scoring='roc_auc')

In [87]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=20,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1600,
              presort='auto', random_state=42, subsample=1, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [88]:
model.best_score_

0.4525011464489949

In [89]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingClassifier(max_depth=20,max_features='sqrt',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=3200,learning_rate=.025),tuned_parameters,cv=5,scoring='roc_auc')

In [90]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.025, loss='deviance', max_depth=20,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=3200,
              presort='auto', random_state=42, subsample=1, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [91]:
model.best_score_

0.44680182953241065

In [92]:
final_model = GradientBoostingClassifier(max_depth=20,max_features='sqrt',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=800,learning_rate=.1)

In [93]:
final_model.fit(x,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=20,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=800,
              presort='auto', random_state=42, subsample=1, verbose=0,
              warm_start=False)

In [94]:
pickle.dump(final_model,open('GBM.pickle','wb'))

# Predictive Model

In [95]:
os.chdir('../../pickles')

In [96]:
x = np.array(df_pca.drop(['Survived'],axis=1))

In [97]:
y = np.array(df_pca['Survived'])

In [98]:
os.chdir('../Models/pca_models')

In [99]:
tuned_parameters={'n_estimators':[100*i for i in range(1,11)]}
model = GridSearchCV(GradientBoostingClassifier(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='roc_auc',verbose=50)

In [100]:
model.fit(x,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.375780 -   0.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.360591 -   0.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.275697 -   0.2s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.621582 -   0.2s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[CV] n_estimators=100 .............................

[CV] ....................... n_estimators=800, score=0.276737 -   1.7s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:   31.2s remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.587026 -   1.4s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   32.6s remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.717196 -   1.4s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   34.0s remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.525073 -   1.7s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:   35.8s remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.219725 -   2.4s
[Parallel(n_jobs=1)]: Done  42 out of  42

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=50)

In [101]:
model.best_score_

0.4769902383074423

In [102]:
model.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False)

In [103]:
tuned_parameters={'n_estimators':[20*i for i in range(1,6)]}
model = GridSearchCV(GradientBoostingClassifier(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='roc_auc',verbose=50)

In [104]:
model.fit(x,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.494902 -   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.373387 -   0.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.355389 -   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.672779 -   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[CV] n_estimators=20 ...............................

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 40, 60, 80, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=50)

In [105]:
model.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.01, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=20,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False)

In [106]:
model.best_score_

0.5583619941321316

In [107]:
tuned_parameters={'max_depth':[8,10,12,15,20],'min_samples_leaf':[.01*i for i in range(1,11)],'subsample':[.8,.9,1]
                 ,'max_features':[None,'sqrt','log2']}
model = GridSearchCV(GradientBoostingClassifier(n_estimators=20,learning_rate=.1,random_state=42),tuned_parameters,cv=5,scoring='roc_auc',verbose=2500)

In [108]:
model.fit(x,y)

Fitting 5 folds for each of 450 candidates, totalling 2250 fits
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.615377 -   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.371203 -   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.354349 -   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.685393 -   0

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03, score=0.564399 -   0.0s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    1.3s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03, score=0.479609 -   0.0s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    1.3s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03, score=0.518935 -   0.0s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    1.4s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03, score=0.716769 -   0.0s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    1.4s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.941895 -   0.0s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    2.2s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.05, score=0.553579 -   0.0s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:    2.2s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.05, score=0.520391 -   0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    2.2s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.05, score=0.698294 -   0.0s
[Parallel(n_jobs=1)]: Done  73 out of  73 | elapsed:    2.3s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.08, score=0.559301 -   0.0s
[Parallel(n_jobs=1)]: Done 111 out of 111 | elapsed:    3.0s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.08 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.08, score=0.693508 -   0.0s
[Parallel(n_jobs=1)]: Done 112 out of 112 | elapsed:    3.1s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.08 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.08, score=0.678943 -   0.0s
[Parallel(n_jobs=1)]: Done 113 out of 113 | elapsed:    3.1s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.08 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.08, score=0.747721 -   0.0s
[Parallel(n_jobs=1)]: Done 114 out of 114 | elapsed:    3.1s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.1, score=0.916810 -   0.0s
[Parallel(n_jobs=1)]: Done 145 out of 145 | elapsed:    3.7s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.1 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.1, score=0.644715 -   0.0s
[Parallel(n_jobs=1)]: Done 146 out of 146 | elapsed:    3.7s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.1 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.1, score=0.812838 -   0.0s
[Parallel(n_jobs=1)]: Done 147 out of 147 | elapsed:    3.7s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.1 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.1, score=0.616105 -   0.0s
[Parallel(n_jobs=1)]: Done 148 out of 148 | elapsed:    3.7s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.1 
[C

...

[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.02, score=0.369330 -   0.0s
[Parallel(n_jobs=1)]: Done 2126 out of 2126 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.02 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.02, score=0.445069 -   0.0s
[Parallel(n_jobs=1)]: Done 2127 out of 2127 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.02 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.02, score=0.438514 -   0.0s
[Parallel(n_jobs=1)]: Done 2128 out of 2128 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.02 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.02, score=0.685181 -   0.0s
[Parallel(n_jobs=1)]: Done 2129 out of 2129 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min

[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.04, score=0.558261 -   0.0s
[Parallel(n_jobs=1)]: Done 2158 out of 2158 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.04 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.04, score=0.712423 -   0.0s
[Parallel(n_jobs=1)]: Done 2159 out of 2159 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.04 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.04, score=0.902980 -   0.0s
[Parallel(n_jobs=1)]: Done 2160 out of 2160 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.05 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.05, score=0.562526 -   0.0s
[Parallel(n_jobs=1)]: Done 2161 out of 2161 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=2

[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07, score=0.586558 -   0.0s
[Parallel(n_jobs=1)]: Done 2191 out of 2191 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07, score=0.674782 -   0.0s
[Parallel(n_jobs=1)]: Done 2192 out of 2192 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07, score=0.725864 -   0.0s
[Parallel(n_jobs=1)]: Done 2193 out of 2193 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.07, score=0.785351 -   0.0s
[Parallel(n_jobs=1)]: Done 2194 out of 2194 | 

[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09, score=0.544112 -   0.0s
[Parallel(n_jobs=1)]: Done 2226 out of 2226 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09, score=0.740637 -   0.0s
[Parallel(n_jobs=1)]: Done 2227 out of 2227 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09, score=0.710674 -   0.0s
[Parallel(n_jobs=1)]: Done 2228 out of 2228 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09, score=0.789485 -   0.0s
[Parallel(n_jobs=1)]: Done 2229 out of 2229 | elapsed:  1.1min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, m

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=20,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'subsample': [0.8, 0.9, 1], 'max_depth': [8, 10, 12, 15, 20], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc',
       verbose=2500)

In [109]:
model.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=20,
              presort='auto', random_state=42, subsample=0.9, verbose=0,
              warm_start=False)

In [110]:
model.best_score_

0.7606493991560825

In [111]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingClassifier(max_depth=8,min_samples_leaf=.1,subsample=.9,random_state=42,n_estimators=40,learning_rate=.05),tuned_parameters,cv=5,scoring='roc_auc')

In [112]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=8,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=40,
              presort='auto', random_state=42, subsample=0.9, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [113]:
model.best_score_

0.7455083759269528

In [114]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingClassifier(max_depth=8,min_samples_leaf=.1,subsample=.9,random_state=42,n_estimators=80,learning_rate=.025),tuned_parameters,cv=5,scoring='roc_auc')

In [115]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.025, loss='deviance', max_depth=8,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=80,
              presort='auto', random_state=42, subsample=0.9, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [116]:
model.best_score_

0.7509207370400591

In [117]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingClassifier(max_depth=8,min_samples_leaf=.1,subsample=.9,random_state=42,n_estimators=10,learning_rate=.2),tuned_parameters,cv=5,scoring='roc_auc')

In [118]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              presort='auto', random_state=42, subsample=0.9, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [119]:
model.best_score_

0.770666436408423

In [120]:
final_model = model.best_estimator_

In [121]:
final_model.fit(x,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              presort='auto', random_state=42, subsample=0.9, verbose=0,
              warm_start=False)

In [122]:
pickle.dump(final_model,open('GBM.pickle','wb'))